In [30]:
import os
import re

import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

import os
from sqlalchemy import create_engine
engine = create_engine(os.getenv('PSQL', None) + 'assembly')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dir_item = '/home/jake/Documents/News_Item/'
dir_proj = dir_item + 'summer_camp/'
dir_tableau = dir_proj + 'tableau_data/'

In [3]:
files = os.listdir(dir_proj)
files

['20-Results.csv', '20CODE_2017.csv', '2017-Money.csv', 'election_result.tsv']

In [4]:
result = pd.read_csv(dir_proj + files[0], encoding='utf8')
code = pd.read_csv(dir_proj + files[1], encoding='euckr')
code['BJD_CD'] = code['BJD_CD'].apply(lambda x: str(x)[:5])
money = pd.read_csv(dir_proj + files[2], encoding='utf8')

In [5]:
code.head()

,BJD_CD,BJD_CD_NM,GU_HJD_CD_NM,CODE,ELEC
0,11110,청운동,종로구_청운효자동,20-001,서울 종로구
1,11110,신교동,종로구_청운효자동,20-001,서울 종로구
2,11110,궁정동,종로구_청운효자동,20-001,서울 종로구
3,11110,효자동,종로구_청운효자동,20-001,서울 종로구
4,11110,창성동,종로구_청운효자동,20-001,서울 종로구


In [6]:
tmp = code.groupby(['BJD_CD']).agg({'CODE':'first'})
tmp.reset_index(inplace=True)
print(tmp.shape)
tmp.head()

(257, 2)


,BJD_CD,CODE
0,11110,20-001
1,11140,20-003
2,11170,20-004
3,11200,20-002
4,11215,20-005


In [7]:
result.head()

,E-Code,Electeded,Number
0,20-001,정세균,2
1,20-002,홍익표,2
2,20-003,지상욱,1
3,20-004,진영,2
4,20-005,전혜숙,2


In [8]:
def transform_number_to_party(x):
    num2party = {
        1: '새누리당',
        2: '더불어민주당',
        3: '국민의당',
        4: '정의당',
        9: '무소속',
    }
    return num2party[x]
merged = result.merge(tmp[['BJD_CD', 'CODE']], left_on='E-Code', right_on='CODE', how='left')
merged['Party'] = merged['Number'].apply(transform_number_to_party)
merged.drop(['E-Code', 'Number',], axis=1, inplace=True)
merged.head()

,Electeded,BJD_CD,CODE,Party
0,정세균,11110,20-001,더불어민주당
1,홍익표,11200,20-002,더불어민주당
2,지상욱,11140,20-003,새누리당
3,진영,11170,20-004,더불어민주당
4,전혜숙,11215,20-005,더불어민주당


In [9]:
merged.head(10)

,Electeded,BJD_CD,CODE,Party
0,정세균,11110,20-001,더불어민주당
1,홍익표,11200,20-002,더불어민주당
2,지상욱,11140,20-003,새누리당
3,진영,11170,20-004,더불어민주당
4,전혜숙,11215,20-005,더불어민주당
5,추미애,NaN,NaN,더불어민주당
6,안규백,11230,20-007,더불어민주당
7,민병두,NaN,NaN,더불어민주당
8,서영교,11260,20-009,더불어민주당
9,박홍근,NaN,NaN,더불어민주당


In [10]:
merged.shape

(337, 4)

In [11]:
# merged.to_csv(dir_proj + 'election_result.tsv', sep='\t', index=False)

# winner vs loser

In [12]:
url = 'https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD_%EC%A0%9C20%EB%8C%80_%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90_%EC%84%A0%EA%B1%B0'
r = requests.get(url)
tables = pd.read_html(r.text)
tables[:2]

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 0  \
 0   NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [13]:
tables[12]

,Unnamed: 0,1위 vs 2위,예측 1위 %,예측 2위 %,편차,"1,2위 후보 비교",실제 결과
0,서울 중구·성동구을,지상욱 (새누리) Vs 정호준 (국민),39.4,33.5,5.9,39.4 33.5,38.0 36.3
1,서울 용산구,진영 (더민주) Vs 황춘자 (새누리),43.7,39.4,4.3,43.7 39.4,42.8 39.9
2,서울 동대문구갑,안규백 (더민주) Vs 허용범 (새누리),41.3,39.5,1.8,41.3 39.5,42.8 38.3
3,서울 광진구갑,전혜숙 (더민주) Vs 정송학 (새누리),42.2,37.2,5.0,42.2 37.2,40.7 37.9
4,서울 강북구갑,정양석 (새누리) Vs 천준호 (더민주),38.4,35.7,2.7,38.4 35.7,39.5 34.7
5,서울 도봉구을,김선동 (새누리) Vs 오기형 (더민주),43.5,37.7,5.8,43.5 37.7,43.7 36.4
6,서울 노원구갑,고용진 (더민주) Vs 이노근 (새누리),42.1,39.1,3.0,42.1 39.1,41.8 39.4
7,서울 양천구을,김용태 (새누리) Vs 이용선 (더민주),41.2,40.6,0.6,41.2 40.6,42.0 39.9
8,서울 강서구갑,금태섭 (더민주) Vs 구상찬 (새누리),36.8,34.4,2.4,36.8 34.4,37.2 32.2
9,서울 금천구,이훈 (더민주) Vs 한인수 (새누리),36.7,34.1,2.6,36.7 34.1,38.1 34.6


In [14]:
len(tables)
tables[26]

,Unnamed: 0,1위 vs 2위,예측 1위 %,예측 2위 %,편차,"1,2위 후보 비교",실제 결과
0,제주 제주시갑,강창일(더민주) Vs 양치석(새누리),46.8,40.1,6.7,46.8 40.1,48.0 36.7
1,제주 제주시을,부상일(새누리) Vs 오영훈(더민주),45.6,43.0,2.6,45.6 43.0,42.3 45.2
2,제주 서귀포시,위성곤(더민주) Vs 강지용(새누리),51.8,48.2,3.6,51.8 48.2,53.5 45.5


In [27]:
score_list = []
for table in tables[12:27]:
    regional_score = []
    for idx, row in table.iterrows():
        candidates = row['1위 vs 2위']
        result = row['실제 결과']
        winner, loser = [re.search(r'(.+)\((.+)\)', c).groups() for c in candidates.split(' Vs ')]
        winner_name, winner_party = [w.strip() for w in winner]
        loser_name, loser_party = [l.strip() for l in loser]
        winner_score, loser_score = [float(c.strip()) for c in result.split(' ')]
        regional_score.append((winner_name, winner_party, loser_name, loser_party, winner_score, loser_score))
    score_list.extend(regional_score)


score = pd.DataFrame(columns=['winner', 'winner_party', 'loser', 'loser_party', 'winner_score', 'loser_score'], data=score_list)
score.head()

,winner,winner_party,loser,loser_party,winner_score,loser_score
0,지상욱,새누리,정호준,국민,38.0,36.3
1,진영,더민주,황춘자,새누리,42.8,39.9
2,안규백,더민주,허용범,새누리,42.8,38.3
3,전혜숙,더민주,정송학,새누리,40.7,37.9
4,정양석,새누리,천준호,더민주,39.5,34.7


In [28]:
# score.to_csv(dir_proj + 'score.tsv', sep='\t', index=False)

# member details

In [37]:
members = pd.read_sql('member_details', engine)
members['jpgName'] = members['jpgLink'].apply(lambda x: x.split('/')[-1])
members.head()

,num,assemEmail,assemTel,bthDate,electionNum,empNm,engNm,examCd,hbbyCd,hjNm,memTitle,origNm,polyNm,reeleGbnNm,secretary,secretary2,shrtNm,staff,deptCd,assemHomep,이름,당선횟수,광역단체,jpgLink,jpgName
0,153,uljukang@assembly.go.kr,02-788-2708,19420605,"17,18,19,20대",강길부,KANG GHILBOO,건설전문가,독서,姜吉夫,"서울대 환경대학원 졸업,\r\n대통령 건설교통 비서관, \r\n건설교통부차관",울산 울주군,무소속,4선제20대,"이수철, 차상윤","고길주, 김변호, 장경희, 박금숙",산업통상자원중소벤처기업위원회,"윤지현, 정재원",9770276,None,강길부,4,울산,http://www.assembly.go.kr/photo/9770276.jpg,9770276.jpg
1,2892,kangbw89@gmail.com,02-784-1422,19710709,20대,강병원,KANG BYUNGWON,None,등산,姜炳遠,"1971년 7월 9일 생 \r\n서울대학교 농경제학과 졸업 (경제학사) \r\n\r\n[경력]\r\n1994 서울대학교 총학생회장\r\n1998 ~ 1999 ㈜대우인터네셔널 무역부문\r\n2002 제16대 대통령선거 노무현 대통령 후보 수행비서\r\n2003 제16대 대통령직 인수위원회 행정관 \r\n2003 ~ 2007 노무현대통령 대통령비서실 청와대 행정관\r\n2011 ㈜글로원씨앤티 대표이사 \r\n2011.1 ~2017.7 &#039사람사는세상 노무현재단&#039 기획위원 \r\n2011 민주당 전략기획위원회 부위원장 \r\n2012.08 ~ 2013.05 민주통합당 홍보위원회 부위원장\r\n2012 제18대 대통령선거 문재인 민주통합당 대통령후보 직능특보실 부실장\r\n2015.04 ~ 2015.12 새정치민주연합 부대변인 \r\n2015.12 ~ 2016.08 더불어민주당 부대변인\r\n2016.03 ~ 더불어민주당 서울 은평(을) 지역위원장\r\n2016.05 ~ 제20대 국회의원 (더불어민주당 / 서울 은평구을)\r\n2016.05 ~ 2017.05 제20대 국회 전반기 국회운영위원회 위원\r\n2016.05 ~ 2018.04 제20대 국회 전반기 환경노동위원회 위원\r\n2016.05 ~ 2017.05 더불어민주당 원내부대표\r\n2016.05 ~ 2016.10 더불어민주당 &lt;가계부채TF&gt; 간사 \r\n2016.10 ~ 더불어민주당 서울시당 을지키는민생실천위원회(을지로위원회) 위원장 \r\n2016.11 ~ 2018.08 더불어민주당 전국청년위원회 운영위원 \r\n2016.12 ~ 2018.08 더불어민주당 환경특별위원회 위원장\r\n2017.03 ~ 2017.05 문재인 더불어민주당 대통령후보 환경노동특보단장, 유세본부 부본부장, 종합상황본부 단장\r\n2017.03 ~ 2018.08 더불어민주당 중앙당 을지키는민생실천위원회(을지로위원회) 운영위원 \r\n2017.06 ~ 2018.08 더불어민주당 정책위원회 부의장 \r\n2017.06 ~ 2018.05 제20대 국회 전반기 예산결산특별위원회 위원\r\n2017.07 ~ 사람사는세상 노무현재단 상임운영위원\r\n2017.07 ~ 2018.08 더불어민주당 생활화학제품안전특별위원회 위원장 \r\n2017.08 ~ 2018.08 더불어민주당 적폐청산위원회 위원\r\n2017.11 헌법재판소장 이진성 후보자 인사청문특별위원회 위원\r\n2017.12 ~ 2018.05 제20대 국회 미세먼지대책 특별위원회 위원\r\n2018.01 ~ 제20대 국회 운영위원회 위원 \r\n2018.05 ~ 더불어민주당 원내대변인\r\n2018.07 ~ 제20대 국회 후반기 기획재정위원회 위원\r\n2018.10 ~ 더불어민주당 자치분권정책협의회 지방혁신균형발전추진단 추진위원\r\n2018.10 ~ 더불어민주당 동북아평화협력특별위원회 위원\r\n2019.02 ~ 더불어민주당 도시재생특별위원회 위원 \r\n\r\n[수상]\r\n2016 더불어민주당 국정감사 우수의원\r\n2016 국회사무처 입법 및 정책개발 최우수 국회의원 \r\n2017 경실련 국정감사 우수의원 20인\r\n2017 한국환경회의 국정감사 환경부문 BEST 의원\r\n2017 국정감사NGO모니터단 우수의원상\r\n2017 한국환경전문기자협회 올해의 환경인상\r\n2017 국회사무처 입법 및 정책개발 우수 국회의원\r\n2018 더불어민주당 국정감사 우수의원상\r\n2019 환경재단 미세먼지센터 제1회 맑은하늘상",서울 은평구을,더불어민주당,초선제20대,"김두현, 김형모","염시진, 김형근, 박화준, 정수현",기획재정위원회,"이여진, 장용득",9770933,http://kangbw.co.kr,강병원,1,서울,http://www.assembly.go.kr/photo/9770933.jpg,9770933.jpg
2,2927,ksjv7900@naver.com,02-784-1460,19591207,20대,강석진,KANG SEOGJIN,None,None,姜錫振,None,경남 산청군함양군거창군합천군,자유한국당,초선제20대,"서일교, 신우성","김준형, 신용욱, 용선주, 정진남",농림축산식품해양수산위원회,"김완철, 오석환",9771036,None,강석진,1,경남,http://www.assembly.go.kr/photo/9771036.jpg,9771036.jpg
3,2788,kangsh@na.go.kr,02-784-2376,19551203,"18,19,20대",강석호,KANG SEOKHO,산악등반,스포츠,姜碩鎬,학력\r\n중동중·고 졸업\r\n한국외국어대학교 스페인어과 졸업\r\n\r\n경력\r\n제20대 국회 외교통일위원장 \r\n제20대 국회 정보위원장\r\n제19대 국회 국토교통위원회 간사\r\n제18대 국회 농림수산식품위원회 간사\r\n새누리당 최고위원\r\n새누리당 제1사무부총장\r\n새누리당 제4정책조정위원장\r\n새누리당 경상북도당 위원장,경북 영양군영덕군봉화군울진군,자유한국당,3선제20대,"김동우, 김정","한승룡, 서재광, 조하나, 박민지",농림축산식품해양수산위원회,"임종석, 홍성범",9770512,http://kangsh.kr,강석호,3,경북,http://www.assembly.go.kr/photo/9770512.jpg,9770512.jpg
4,155,kang809@gmail.com,02-784-6084,19520128,"17,18,19,20대",강창일,KANG CHANGIL,바둑,독서.음악감,姜昌一,"17·18·19·20대 국회의원\r\n서울대학교 국사학과·동경대학 문학부 석·박사(문학박사)\r\n한·일의원연맹 회장\r\n국회 한·몽골 의원친선협회 회장 \r\n국회 행정안전위원회 위원 \r\n국회 민주주의와 복지국가연구회 공동대표\r\n국회 외교통일위원회·예산결산특위·개헌특위 위원(전)\r\n새정치민주연합 윤리심판원 원장(전)\r\n국회 산업통상자원위원회·지식경제위원회 위원장(전)\r\n국회 독도영토수호대책특위 위원장(전)\r\n배재대 교수, 제주4·3연구소장(전)",제주 제주시갑,더불어민주당,4선제20대,"김명언, 유동현","배경민, 정민희, 하소라, 이진욱, 신윤정",행정안전위원회,"강신혁, 김행준",9770279,http://www.kangci.net,강창일,4,제주,http://www.assembly.go.kr/photo/9770279.jpg,9770279.jpg


In [38]:
# members[['empNm', 'jpgName', 'assemTel', 'assemEmail', 'electionNum', 'hbbyCd', 'origNm']].to_csv(dir_proj + 'member_details.tsv', sep='\t', index=False)